In [5]:
from langchain.document_loaders.pdf import PyPDFDirectoryLoader

def load_documents():
    document_loader = PyPDFDirectoryLoader('./pdf_data')
    return document_loader.load()

In [6]:
documents = load_documents()
print(documents[0])

page_content='FM 3-50
ARMY PERSONNEL 
RECOVERY
SEPTEMBER 2014
DISTRIBUTION RESTRICTION:
Approved for public release; distribution is unlimited.
HEADQUARTERS, DEPARTMENT OF THE ARMY' metadata={'source': 'pdf_data/fm3_50.pdf', 'page': 0}


In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema.document import Document

def split_documents(documents : list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, 
            chunk_overlap=300,
            length_function=len,
            is_separator_regex=False,
            )
    return text_splitter.split_documents(documents)



In [8]:
chunks = split_documents(documents)
chunks

[Document(metadata={'source': 'pdf_data/fm3_50.pdf', 'page': 0}, page_content='FM 3-50\nARMY PERSONNEL \nRECOVERY\nSEPTEMBER 2014\nDISTRIBUTION RESTRICTION:\nApproved for public release; distribution is unlimited.\nHEADQUARTERS, DEPARTMENT OF THE ARMY'),
 Document(metadata={'source': 'pdf_data/fm3_50.pdf', 'page': 1}, page_content='This publication is available at Army Knowledge Online \n(https://armypubs.us.army.mil/doctrine/index.html). \nTo receive publishing updates, please subscribe at \nhttp://www.apd.army.mil/AdminPubs/new_subscribe.asp.'),
 Document(metadata={'source': 'pdf_data/fm3_50.pdf', 'page': 2}, page_content='*FM 3-50 \nDistribution Restriction: Approved for public release; distribution is unlimited. \n*This publication supersedes FM 3-50.1, 21 November 2011. \ni \n \nField Manual \nNo. 3-50 \nHeadquarters \nDepartment of the Army \nWashington, DC, 2 September 2014 \nArmy Personnel Recovery \nContents \nPage \n PREFACE....................................................

In [9]:
from langchain_ollama import OllamaEmbeddings
def get_embedding_function():
    embeddings = OllamaEmbeddings(model="nomic-embed-text")
    return embeddings 


In [10]:
from langchain_chroma import Chroma
def add_to_chroma(chunks: list[Document]):
    unique_ids = []
    prev = None
    index = 0
    for chunk in chunks:
        source = chunk.metadata.get("source")
        page = chunk.metadata.get("page")
        curr_id = f"{source}:{page}"
        
        if curr_id == prev:
            index += 1
        else:
            index = 0
        unique_id = f"{curr_id}:{index}"
        unique_ids.append(unique_id)
        prev = curr_id

    db = Chroma(
        persist_directory="./chroma_db", 
        embedding_function=get_embedding_function()
    )
    
    db.add_documents(chunks, ids=unique_ids)
    print(f"Added {len(chunks)} documents to Chroma and persisted to {'./chroma_db'}")


In [14]:
add_to_chroma(chunks)

AttributeError: 'Chroma' object has no attribute 'persist'

In [19]:
from llama_index.llms.ollama import Ollama

def query_rag(query: str):
    db = Chroma(
        persist_directory="./chroma_db", 
        embedding_function=get_embedding_function()
    )
    results = db.similarity_search_with_score(query, k=3)
    context = "\n".join([result[0].page_content for result in results])
    return context

llm = Ollama(model="hf.co/bartowski/Llama-3.2-3B-Instruct-GGUF:latest", request_timeout=120.0)

def chat_with_bot(user_question: str):
    context = query_rag(user_question)
    prompt = f"""
    You are a helpful assistant. Use the context below to answer the user's question.

    Context:
    {context}

    User: {user_question}

    Assistant: Let's think about this step by step to provide a clear and concise answer.
    """
    response = llm.complete(prompt)
    return response

# Example usage
user_input = "Tell me how advancements in AI can help the army give me three ways"
response = chat_with_bot(user_input)
print(response)

Based on the context provided, I'll break down how advancements in AI can help the Army. Here are three ways:

1. **Enhanced Situational Understanding**: AI can analyze vast amounts of data from various sources, including sensors, satellites, and other intelligence platforms. This enables the Army to gain a deeper understanding of the operational environment, identify patterns, and predict enemy movements. By leveraging AI-driven analytics, the Army can make more informed decisions and seize opportunities that might otherwise be missed.

2. **Improved Multidomain Operations**: AI can help synchronize joint and Army capabilities across multiple domains (e.g., air, land, sea), allowing for more effective and flexible operations. AI-driven systems can analyze and integrate data from various sources in real-time, providing commanders with a unified view of the battlefield. This enables the Army to exploit relative advantages, defeat enemy forces, and consolidate gains.

3. **Advanced Infor

In [ ]:
from llama_index.llms.ollama import Ollama

class ChatbotMemory:
    def __init__(self):
        self.conversation_history = []

    def add_to_memory(self, user_input, assistant_response):
        self.conversation_history.append((user_input, assistant_response))

    def get_memory(self):
        return "\n".join([f"User: {user}\nAssistant: {assistant}" for user, assistant in self.conversation_history])

def query_rag(query: str):
    db = Chroma(
        persist_directory="./chroma_db", 
        embedding_function=get_embedding_function()
    )
    results = db.similarity_search_with_score(query, k=3)
    context = "\n".join([result[0].page_content for result in results])
    return context

llm = Ollama(model="hf.co/bartowski/Llama-3.2-3B-Instruct-GGUF:latest", request_timeout=120.0)
memory = ChatbotMemory()

def chat_with_bot(user_question: str):
    context = query_rag(user_question)
    memory_context = memory.get_memory()
    prompt = f"""
    You are a helpful assistant. Use the context below to answer the user's question.

    Context:
    {context}

    Conversation History:
    {memory_context}

    User: {user_question}

    Assistant: Let's think about this step by step to provide a clear and concise answer.
    """
    response = llm.complete(prompt)
    memory.add_to_memory(user_question, response)
    return response

# Example usage
user_input = "Tell me how advancements in AI can help the army give me three ways"
response = chat_with_bot(user_input)
print(response)